In [31]:
import os 
import sys
from dotenv import load_dotenv
import json
import time
import random
import tiktoken

import pandas as pd
from typing import List
from beautifultable import BeautifulTable
import camelot

from langchain_community.document_loaders import CSVLoader
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import lxml
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

In [32]:
file_path = "../data/Mcdonald_review - 6 months - 40 stores.csv"

In [36]:
df = pd.read_csv(file_path)


## Data Preprocessing

In [37]:
# Define a mapping for star ratings
rating_map = {
    '1 star': 1.0,
    '2 stars': 2.0,
    '3 stars': 3.0,
    '4 stars': 4.0,
    '5 stars': 5.0
}

# Map ratings using the defined dictionary
df['rating'] = df['rating'].replace(rating_map)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with NaN ratings
df = df.dropna(subset=['rating'])

# Convert latitude and longitude to numeric
df['latitude'] = pd.to_numeric(df['latitude '], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df = df.drop(columns= ['latitude ','reviewer_id','longitude','latitude','review'])


C:\Users\York Yong\AppData\Local\Temp\ipykernel_10304\4248795631.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['rating'] = df['rating'].replace(rating_map)


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3492 entries, 0 to 3491
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   store_address  3492 non-null   object 
 1   rating_count   3492 non-null   object 
 2   review_time    3492 non-null   object 
 3   rating         3492 non-null   float64
 4   sentiment      3492 non-null   object 
dtypes: float64(1), object(4)
memory usage: 136.5+ KB


## Convert to multiple formats for experiments

In [39]:
# prepare test set
eval_df = pd.DataFrame(columns=["Data Format", "Data raw"]) # , "Question", "Answer"

In [40]:
# Save the data in JSON format
data_json = df.to_json(orient='records')
eval_df.loc[len(eval_df)] = ["JSON", data_json]

# Save the data as a list of dictionaries
data_list_dict = df.to_dict(orient='records')
eval_df.loc[len(eval_df)] = ["DICT", data_list_dict]

# Save the data in CSV format
csv_data = df.to_csv(index=False)
eval_df.loc[len(eval_df)] = ["CSV", csv_data]

# Save the data in tab-separated format
tsv_data = df.to_csv(index=False, sep='\t')
eval_df.loc[len(eval_df)] = ["TSV (tab-separated)", tsv_data]

# Save the data in HTML format
html_data = df.to_html(index=False)
eval_df.loc[len(eval_df)] = ["HTML", html_data]

# Save the data in LaTeX format
latex_data = df.to_latex(index=False)
eval_df.loc[len(eval_df)] = ["LaTeX", latex_data]

# Save the data in Markdown format
markdown_data = df.to_markdown(index=False)
eval_df.loc[len(eval_df)] = ["Markdown", markdown_data]

# Save the data as a string
string_data = df.to_string(index=False)
eval_df.loc[len(eval_df)] = ["STRING", string_data]

# Save the data as a NumPy array
numpy_data = df.to_numpy()
eval_df.loc[len(eval_df)] = ["NumPy", numpy_data]

# Save the data in XML format
xml_data = df.to_xml(index=False)
eval_df.loc[len(eval_df)] = ["XML", xml_data]

In [41]:
from pandas import option_context
with option_context('display.max_colwidth', 500):
    display(eval_df.head(10))

,Data Format,Data raw
0,JSON,"[{""store_address"":""13749 US-183 Hwy, Austin, TX 78750, United States"",""rating_count"":""1,240"",""review_time"":""3 months ago"",""rating"":1.0,""sentiment"":""negative""},{""store_address"":""13749 US-183 Hwy, Austin, TX 78750, United States"",""rating_count"":""1,240"",""review_time"":""5 days ago"",""rating"":4.0,""sentiment"":""positive""},{""store_address"":""13749 US-183 Hwy, Austin, TX 78750, United States"",""rating_count"":""1,240"",""review_time"":""5 days ago"",""rating"":1.0,""sentiment"":""negative""},{""store_address"":""13749 U..."
1,DICT,"[{'store_address': '13749 US-183 Hwy, Austin, TX 78750, United States', 'rating_count': '1,240', 'review_time': '3 months ago', 'rating': 1.0, 'sentiment': 'negative'}, {'store_address': '13749 US-183 Hwy, Austin, TX 78750, United States', 'rating_count': '1,240', 'review_time': '5 days ago', 'rating': 4.0, 'sentiment': 'positive'}, {'store_address': '13749 US-183 Hwy, Austin, TX 78750, United States', 'rating_count': '1,240', 'review_time': '5 days ago', 'rating': 1.0, 'sentiment': 'negativ..."
2,CSV,"store_address,rating_count,review_time,rating,sentiment\r\n""13749 US-183 Hwy, Austin, TX 78750, United States"",""1,240"",3 months ago,1.0,negative\r\n""13749 US-183 Hwy, Austin, TX 78750, United States"",""1,240"",5 days ago,4.0,positive\r\n""13749 US-183 Hwy, Austin, TX 78750, United States"",""1,240"",5 days ago,1.0,negative\r\n""13749 US-183 Hwy, Austin, TX 78750, United States"",""1,240"",a month ago,5.0,positive\r\n""13749 US-183 Hwy, Austin, TX 78750, United States"",""1,240"",2 months ago,1.0,negative\..."
3,TSV (tab-separated),"store_address\trating_count\treview_time\trating\tsentiment\r\n13749 US-183 Hwy, Austin, TX 78750, United States\t1,240\t3 months ago\t1.0\tnegative\r\n13749 US-183 Hwy, Austin, TX 78750, United States\t1,240\t5 days ago\t4.0\tpositive\r\n13749 US-183 Hwy, Austin, TX 78750, United States\t1,240\t5 days ago\t1.0\tnegative\r\n13749 US-183 Hwy, Austin, TX 78750, United States\t1,240\ta month ago\t5.0\tpositive\r\n13749 US-183 Hwy, Austin, TX 78750, United States\t1,240\t2 months ago\t1.0\tnegat..."
4,HTML,"<table border=""1"" class=""dataframe"">\n <thead>\n <tr style=""text-align: right;"">\n <th>store_address</th>\n <th>rating_count</th>\n <th>review_time</th>\n <th>rating</th>\n <th>sentiment</th>\n </tr>\n </thead>\n <tbody>\n <tr>\n <td>13749 US-183 Hwy, Austin, TX 78750, United States</td>\n <td>1,240</td>\n <td>3 months ago</td>\n <td>1.0</td>\n <td>negative</td>\n </tr>\n <tr>\n <td>13749 US-183 Hwy, Austin, TX 78750, U..."
5,LaTeX,"\begin{tabular}{lllrl}\n\toprule\nstore_address & rating_count & review_time & rating & sentiment \\\n\midrule\n13749 US-183 Hwy, Austin, TX 78750, United States & 1,240 & 3 months ago & 1.000000 & negative \\\n13749 US-183 Hwy, Austin, TX 78750, United States & 1,240 & 5 days ago & 4.000000 & positive \\\n13749 US-183 Hwy, Austin, TX 78750, United States & 1,240 & 5 days ago & 1.000000 & negative \\\n13749 US-183 Hwy, Austin, TX 78750, United States & 1,240 & a month ago & 5.000000 & positi..."
6,Markdown,"| store_address | rating_count | review_time | rating | sentiment |\n|:---------------------------------------------------------------------------|:---------------|:--------------|---------:|:------------|\n| 13749 US-183 Hwy, Austin, TX 78750, United States | 1,240 | 3 months ago | 1 | negative |\n| 13749 US-183 Hwy, Austin, TX 78750, United States | 1,2..."
7,STRING,"store_address rating_count review_time rating sentiment\n 13749 US-183 Hwy, Austin, TX 78750, United States 1,240 3 months ago 1.0 negative\n 13749 US-183 Hwy, Austin, TX 78750, United States 1,240 5 days ago 4.0 positive\n 13749 US-183 Hwy, Austin, TX 78750, United States 1,240 5 days ago 1.0 negative\n ..."
8,NumPy,"[[13749 US-183 Hwy, Austin, TX 78750, United States, 1,240, 3 months ago, 1.0, negative], [13749 US-183 Hwy, Austin, TX 78750, United States, 1,240, 5 days ago, 4.0, positive], [13749 US-183 Hwy,

## Set model for validation

In [44]:
MESSAGE_SYSTEM_CONTENT = """You are a customer service agent that helps a customer with answering questions. 
Please answer the question based on the provided context below. 
Make sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. 
If the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer."""

In [45]:
from langchain_groq import ChatGroq

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [46]:
chat = ChatGroq(
    groq_api_key=GROQ_API_KEY, 
    model_name='llama-3.1-70b-versatile')


In [47]:
# Token counter
def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Rate limiter
class RateLimiter:
    def __init__(self, tokens_per_minute):
        self.tokens_per_minute = tokens_per_minute
        self.tokens_used = 0
        self.last_reset_time = time.time()

    def wait_if_needed(self, tokens):
        current_time = time.time()
        time_passed = current_time - self.last_reset_time
        
        if time_passed >= 60:
            self.tokens_used = 0
            self.last_reset_time = current_time
        
        if self.tokens_used + tokens > self.tokens_per_minute:
            sleep_time = 60 - time_passed
            print(f"Rate limit approaching. Sleeping for {sleep_time:.2f} seconds.")
            time.sleep(sleep_time)
            self.tokens_used = 0
            self.last_reset_time = time.time()
        
        self.tokens_used += tokens

rate_limiter = RateLimiter(tokens_per_minute=19999)  # Adjust as needed

In [48]:
def response_test(question: str, context: str):
    messages = [
        SystemMessage(content=MESSAGE_SYSTEM_CONTENT),
        HumanMessage(content=question),
        AIMessage(content=context)
    ]
    
    # Count tokens
    total_tokens = sum(num_tokens_from_string(msg.content) for msg in messages)
    
    # Wait if needed to avoid rate limit
    rate_limiter.wait_if_needed(total_tokens)
    
    response = chat(messages)
    return response.content

def run_question_test(query: str, eval_df: pd.DataFrame):
    questions = []
    answers = []
    for index, row in eval_df.iterrows():
        questions.append(query)
        response = response_test(query, str(row['Data raw']))
        answers.append(response)
    eval_df['Question'] = questions
    eval_df['Answer'] = answers
    return eval_df

def BeautifulTableformat(query: str, results: pd.DataFrame, MaxWidth: int = 250):
    table = BeautifulTable(maxwidth=MaxWidth, default_alignment=BeautifulTable.ALIGN_LEFT)
    table.columns.header = ["Data Format", "Query", "Answer"]
    for index, row in results.iterrows():
        table.rows.append([row['Data Format'], query, row['Answer']])
    return table

## Test with query

In [49]:
query = "How many stores are there in the dataset?"
result_df1 = run_question_test(query, eval_df.copy())
table = BeautifulTableformat(query, result_df1, 150)
print(table)

Rate limit approaching. Sleeping for 57.36 seconds.
